In [1]:
using Gurobi, JuMP, DataFrames, CSV, Plots

# Utility Function

In [ ]:
function distance(x1, y1, x2, y2)
    floatpointcorrection = sin(y1)*sin(y2) + cos(y1)*cos(y2)*cos(x1-x2)
    floatpointcorrection = ifelse(floatpointcorrection > 1, 1.0, floatpointcorrection)
    floatpointcorrection = ifelse(floatpointcorrection < 0, 0.0, floatpointcorrection)
    ß = acos(floatpointcorrection)
    return ß * 6371
end;

# Importing data, rough edits very quickly to clean it up

In [3]:
data = CSV.read("./data/final_prod.csv", DataFrame)
select!(data, Not(:Column1))
select!(data, Not(["Area Abbreviation", "Area Code", "Unit", "Item Code"]))
data = data[data[:,7] .!= "Miscellaneous", :]
data = data[data[:,7] .!= "Aquatic Products, Other", :]
population = data[:, [:Area, :population]];
food = data[1, 1:5];

# Partitioning and formatting data for Gurobi

In [4]:
demand = combine(groupby(data[!,1:6], :Area), first)
for row in eachrow(demand)
    row[:fats] = 365 * row[:fats]
    row[:proteins] = 365 * row[:proteins]
    row[:carbs] = 365 * row[:carbs]
    row[:fruits_and_veggies] = 365 * row[:fruits_and_veggies]
end
demand = Matrix(demand[:,2:5]);

In [5]:
loc = unique(data[!,[6,8,9]]);

In [6]:
supply = data[:,[6,7,10]]
supplyT = DataFrame(zeros((length(unique(supply[:,1])), length(unique(supply[:,2])))), :auto)
rename!(supplyT, unique(supply[:,2]));
supplyg = groupby(supply, :Area)
for i in 1:168
    for row in eachrow(supplyg[i])
        supplyT[i, Symbol(row[2])] = row[3]
    end
end

In [ ]:
inverse_populations = inv.(unique(population)[!,2])
weighting = CSV.read("./data/weighting.csv", DataFrame)
weighting = Matrix(weighting[:,2:end])
n,p1 = size(supplyT)
n,p2 = size(demand)
lambda = 250*365;
arcDistance = DataFrame(zeros(n, n), :auto)
for i in 1:168
    for j in 1:168
        arcDistance[i, j] = distance(loc[i,2], loc[i,3], loc[j,2], loc[j,3])
    end
end
arcDistance = Matrix(round.(arcDistance, digits=7));

# Optimization

In [12]:
å=1

1

Think of å as the proportion of importance placed on the cost of people going hungry relative to the cost of transportation. (i.e., we only consider transportation cost at 10% the magnitude to which we care feeding people)

In [13]:
model = Model(Gurobi.Optimizer )
@variable(model, X[i=1:n,j=1:n,k=1:p1] .≥ 0)
@variable(model, ß ≥ 0, integer=true)
@constraint(model, realSupply[i=1:n,k=1:p1], sum(X[i,j,k] for j=1:n) ≤ abs.(supplyT)[i,k])
@constraint(model, auxiliary, ß ≥ inverse_populations' * (demand - sum(X[i,:,:] for i=1:n) * 9072000 * weighting) * [1/41.3, 1/107.38, 1/37.9961, 1/330.4])
@constraint(model, realDemand, sum(X[i,:,:] for i=1:n) * 9072000 * weighting .≤ demand)
@objective(model, Min, (1-å) * sum(sum(arcDistance .* sum(X[:,:,k] for k=1:p1), dims=1)) + (å) * ß * lambda)
optimize!(model)

In [11]:
JuMP.value.(ß)

140.0

In [ ]:
JuMP.value.(ß)

In [ ]:
sum(sum(arcDistance .* sum(X[:,:,k] for k=1:p1), dims=1))